In [6]:
import boto3
import awswrangler as wr

In [57]:
df1 = wr.athena.read_sql_query('''
SELECT campaign_id, 
    SUM(cost) AS cost, 
    SUM(impressions) AS impressions, 
    SUM(clicks) AS clicks,
    SUM(cost) / SUM(clicks)   AS cpc
FROM 
    sp_traffic 
WHERE 
    match_type IN('TARGETING_EXPRESSION_PREDEFINED','TARGETING_EXPRESSION') 
    AND cast(from_iso8601_timestamp(time_window_start) as date) >  date_add('day',-5,current_date) 
GROUP BY 
    campaign_id 
HAVING
    SUM(cost) / SUM(clicks) > 0.6''', database="stream_sp_traffic")

In [58]:
print(df1.to_string())

       campaign_id   cost  impressions  clicks    cpc
0  185402650650808  10.02         3590      15  0.668


In [192]:
######### SNS Budget ##########

# use sns subscription
def subscribeSns(snsClient, endpoint):
    response = snsClient.subscribe(
        TopicArn='arn:aws:sns:us-east-1:521981389199:stream_budget',
        Protocol='sqs',
        Endpoint=endpoint,
        ReturnSubscriptionArn=True,
        Attributes={
            'RawMessageDelivery': 'True'
        }
    )
    print (response)

def subscribe_to_sns(snsClient, accounts):
    prefix = 'arn:aws:sqs:us-east-1:'
    suffix = ':budget-usage'
#    suffix = ':testqueue'

    for i in range(len(accounts)):
        endpoint = prefix + accounts[i] + suffix
        print (endpoint)
        subscribeSns(snsClient, endpoint)
        
snsClient = boto3.client('sns')

#accounts = ['521981389199','551344820358']
accounts = ['551344820358']

#accounts = ['521981389199']
            
subscribe_to_sns(snsClient, accounts)

SyntaxError: expression cannot contain assignment, perhaps you meant "=="? (1560536985.py, line 8)

In [197]:
# send SNS message
def send_sns_msg(message):
    response = snsClient.publish(
        TopicArn='arn:aws:sns:us-east-1:521981389199:stream_budget',
        Message= message
    )
    print (response)
    
# read from budget sample data
sample_budget_file = open('/Users/hwenjing/workspace/amz-marketing-stream-workshop/sample-data/budget-usage-data.txt', 'r')
  
sample_budget = []
# Using for loop
for line in sample_budget_file:
    sample_budget.append(line)

sample_budget_file.close()


for i in range(len(sample_budget)):
    for j in range (len(accounts)):
        send_sns_msg(sample_budget[i])


{'MessageId': 'b2d91b67-358d-5585-9e97-40ae980c464f', 'ResponseMetadata': {'RequestId': 'dba43a6d-795e-5f79-8c20-61a4d4dfecb3', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'dba43a6d-795e-5f79-8c20-61a4d4dfecb3', 'content-type': 'text/xml', 'content-length': '294', 'date': 'Wed, 19 Oct 2022 07:46:08 GMT'}, 'RetryAttempts': 0}}
{'MessageId': '1f5752df-2170-53a5-bc2d-09168cf1706f', 'ResponseMetadata': {'RequestId': '94d68949-fa8e-5ed8-9081-d280570f5d12', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '94d68949-fa8e-5ed8-9081-d280570f5d12', 'content-type': 'text/xml', 'content-length': '294', 'date': 'Wed, 19 Oct 2022 07:46:08 GMT'}, 'RetryAttempts': 0}}
{'MessageId': 'b3978acb-77cd-50fb-b08b-d102da4f2331', 'ResponseMetadata': {'RequestId': '0d9ea2a9-5c78-52e9-89b3-f960f1fa7123', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '0d9ea2a9-5c78-52e9-89b3-f960f1fa7123', 'content-type': 'text/xml', 'content-length': '294', 'date': 'Wed, 19 Oct 2022 07:46:10

In [199]:
######### SNS sp-traffic ##########

# use sns subscription
def subscribeTrafficSns(snsClient, endpoint):
    response = snsClient.subscribe(
        TopicArn='arn:aws:sns:us-east-1:521981389199:stream_traffic',
        Protocol='sqs',
        Endpoint=endpoint,
        ReturnSubscriptionArn=True,
        Attributes={
            'RawMessageDelivery': 'True'
        }
    )
    print (response)

def subscribe_to_sns(snsClient, accounts):
    prefix = 'arn:aws:sqs:us-east-1:'
#    suffix = ':budget-usage'
    suffix = ':sp-traffic-mock'

    for i in range(len(accounts)):
        endpoint = prefix + accounts[i] + suffix
        print (endpoint)
        subscribeTrafficSns(snsClient, endpoint)
        
snsClient = boto3.client('sns')

#accounts = ['521981389199','551344820358']
#accounts = ['551344820358']

accounts = ['521981389199']
            
subscribe_to_sns(snsClient, accounts)


arn:aws:sqs:us-east-1:521981389199:sp-traffic-mock
{'SubscriptionArn': 'arn:aws:sns:us-east-1:521981389199:stream_traffic:20de777d-9e0e-45fd-a61c-2a0f28962a90', 'ResponseMetadata': {'RequestId': '9d9d4250-1c76-55a5-8982-1999dd98a4ec', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '9d9d4250-1c76-55a5-8982-1999dd98a4ec', 'content-type': 'text/xml', 'content-length': '364', 'date': 'Wed, 19 Oct 2022 09:13:17 GMT'}, 'RetryAttempts': 0}}


In [200]:
# send SNS message
def send_traffic_sns_msg(message):
    response = snsClient.publish(
        TopicArn='arn:aws:sns:us-east-1:521981389199:stream_traffic',
        Message= message
    )
    print (response)
    
# read from budget sample data
sample_traffic_file = open('/Users/hwenjing/workspace/amz-marketing-stream-workshop/sample-data/sp-traffic-data.txt', 'r')
  
sample_traffic = []
# Using for loop
for line in sample_traffic_file:
    sample_traffic.append(line)

sample_traffic_file.close()


for i in range(len(sample_traffic)):
    for j in range (len(accounts)):
        send_traffic_sns_msg(sample_traffic[i])


{'MessageId': '53ed946d-2432-51a2-8926-cfa4e63f1982', 'ResponseMetadata': {'RequestId': 'fd3ebdd1-3fef-516d-8525-e618fde2594d', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'fd3ebdd1-3fef-516d-8525-e618fde2594d', 'content-type': 'text/xml', 'content-length': '294', 'date': 'Wed, 19 Oct 2022 09:13:48 GMT'}, 'RetryAttempts': 0}}
{'MessageId': 'dfc54a07-fe8b-5f11-9b39-11ce3c867731', 'ResponseMetadata': {'RequestId': '62ced103-1a5b-5520-82ea-8acad856ad00', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '62ced103-1a5b-5520-82ea-8acad856ad00', 'content-type': 'text/xml', 'content-length': '294', 'date': 'Wed, 19 Oct 2022 09:13:48 GMT'}, 'RetryAttempts': 0}}
{'MessageId': '5d3aa4f8-4a1d-514b-a673-ff318813e96b', 'ResponseMetadata': {'RequestId': 'bf5eac8c-60dc-50c8-8f58-f745a2d2a8e8', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'bf5eac8c-60dc-50c8-8f58-f745a2d2a8e8', 'content-type': 'text/xml', 'content-length': '294', 'date': 'Wed, 19 Oct 2022 09:13:49